In [7]:
import os
import numpy as np
import pandas as pd
import netCDF4 as nc
import datetime as dt
import geopy.distance
import xarray as xr
import cfgrib

existing_years_era5 = [str(year) for year in range(2023, 2026)]

existing_months_era5_2023 = [
    '01', '02', '03',
    '04', '05', '06',
    '07', '08', '09',
    '10', '11', '12',
]

existing_months_era5_2024 = [
    '01', '02', '03',
    '04', '05', '06',
    '07', '08', '09',
    '10', '11', '12'
]

existing_months_era5_2025 = [
    '01', '02', '03', '04'
]

#existing_months_era5 = ['01', '02'] #shorter, for debug

colsorder_weather=['year',
 'month',
 'day',
 'hour',
 'mean_temp',
 'wind_speed',
 'wind_direction',
 'precip',
    'cloud_coverage', 
                   'atm_pressure'
 #'soiltemp',
 #'snowdepth',
 #'humidity',
 #'radiation'
 ]


In [8]:
def get_param_era5(filepath, suffix, lat, long, yearfrom, yearto, paramname, hourly_flag):
    #outdf=pd.DataFrame({'time':[], paramname:[]}).set_index('time', inplace=True)
    outdf=[pd.DataFrame({'time':[], paramname:[]}) for i in range(len(lat))]
    for year in range(yearfrom, yearto+1):
        if (existing_years_era5.count(str(year))<1):
            print ('Year '+str(year)+ ' is not downloaded')
            exit(-1)
        else:
            #for month in existing_months_era5:
            if (year == 2023): existing_months_era5 = existing_months_era5_2023
            if (year == 2024): existing_months_era5 = existing_months_era5_2024
            if (year == 2025): existing_months_era5 = existing_months_era5_2025
            for month in existing_months_era5:
                print ('reading '+ suffix+'_'+str(year)+'_'+month+'.grib')
                ds = xr.load_dataset(os.path.join(filepath, suffix+'_'+str(year)+'_'+month+'.grib'), engine="cfgrib", chunks={"latitude":500, "longitude":500, "time": -1, "step": -1})
                #ds = ds.interpolate_na()
                
                data_name = list (ds.keys())[0]
                # the problem is that there are cases where we have step as a dimensions and sometimes we don't
                # let's deal with that
                if (len(ds.dims)==4):
                    ds =ds.stack(real_time=('time', 'step')).swap_dims({'real_time':'valid_time'})
                else:
                    ds =ds.swap_dims({'time':'valid_time'})
                for count, (lati, longi) in enumerate(zip(lat,long)):    
                    #out =ds.sel(latitude=lati, longitude=longi,method='nearest',drop=True).resample(valid_time='1D').mean(dim='valid_time').to_dataframe()
                    if(hourly_flag):
                        out =ds.sel(latitude=lati, longitude=longi,method='nearest',drop=True).to_dataframe()
                    else:
                        out =ds.sel(latitude=lati, longitude=longi,method='nearest',drop=True).resample(valid_time='1D').mean(dim='valid_time').to_dataframe()                        
                    out.rename(columns={data_name:paramname}, inplace=True)
                    out.drop(out.columns.difference([paramname]), axis=1, inplace=True)
                    #out.dropna(inplace=True)
                    out.index.names = ['time']
                    out.reset_index(inplace=True)
                    #out.plot()
                    #plt.show()
                    out['time']=pd.to_datetime(out['time'], utc = True)
                    out.drop(out[out['time']<dt.datetime(year, int(month), 1, 0, 0, 0, tzinfo=dt.timezone.utc)].index, inplace=True)
                    if (int(month)<12):
                        out.drop(out[out['time']>=dt.datetime(year, int(month)+1, 1, 0, 0, 0, tzinfo=dt.timezone.utc)].index, inplace=True)
                    if (int(month)==12):
                        out.drop(out[out['time']>=dt.datetime(year+1, 1, 1, 0, 0, 0, tzinfo=dt.timezone.utc)].index, inplace=True)
                    outdf[count]=pd.concat([outdf[count], out], axis=0, ignore_index=True)
    return outdf


In [9]:
def interpolate_data(input_df):
    replaced=input_df.replace(-9999, np.nan) # first replace all -9999 by nan if there are
    # check if the entire column is nan--> no point to interpolate
    for i in replaced.columns:
        if (replaced[i].isnull().all()):
            # replace it back --> will be ignored
            replaced[i]=replaced[i].replace(np.nan,-9999).astype(np.int64)
    replaced.interpolate(method='linear', limit_direction='both', axis=0, inplace=True)
    return replaced


In [10]:
def write_weather(df_towrite, filename):
    print("Writing "+filename)
    linesnum=len(df_towrite)
    with open(filename, 'w',newline=None) as fd:
        fd.write('2'+"\n")#version number, means hourly
        fd.write( str(linesnum)+"\n")
    # let us 
    df_towrite=df_towrite[colsorder_weather]
    df_towrite.to_csv(filename, sep='\t', index = False, header = True, mode='a+', float_format='%.3f')


In [11]:
def country_name_coordinate_year():
    
    
    # Fot bats
    startyear = 2023
    endyear = 2025
   
    # Bat data

    ys = [55.60530,
            55.70201,
            55.79587,
            55.73477,
            55.88947,
            55.79229,
            55.89259,
            55.98644,
            56.08316,
            56.17700,
            56.08137,
            56.17340,
            56.16899,
            55.88101,
            55.97842,
            55.88547,
            55.98287,
            56.16338,
            56.35936,
            56.27060,
            56.36755,
            56.27373,
            55.74873335,
            56.12676,
            55.89034,
            55.81821,
            55.697319,
            55.687427,
            55.688484,
            55.68251,
            55.70688,
            55.74106,
            55.72575,
            55.69058,
            55.71823,
            55.74106,
            55.73732,
            55.72136,
         55.3403,
55.5570,
55.9127,
56.1587,
56.2787,
55.7965,
55.3986,
55.23715,
57.7473,
55.2802,
55.0678,
55.661573,
55.668334,
55.673155,
55.663637,
         55.6903,
         55.1217,55.711433,
55.660532
]
    xs = [7.88513,
            8.05116,
            7.87969,
            7.79082,
            7.70791,
            7.53970,
            8.04651,
            7.87419,
            8.04181,
            7.86861,
            7.70145,
            7.52514,
            7.18712,
            7.02863,
            7.19602,
            7.36837,
            7.53255,
            6.84414,
            7.17649,
            7.69458,
            7.86298,
            8.03705,
            7.901,
            7.43632,
            7.0423,
            7.70747,
            7.571899,
            7.573644,
            7.601428,
            7.77294,
            7.7086,
            7.68745,
            7.553378,
            7.657018,
            7.599585,
            7.68745,
            7.575647,
            7.683028,
         8.6773,
8.0867,
8.4048,
8.1458,
8.1706,
8.2123,
8.4230,
8.7078,
10.6027,
8.5393,
8.6802,
7.668805,
7.694158,
7.719361,
7.724361,
         7.6863,
         8.5044,
         7.682502,
7.641038
]
    
    
    names = ["Loc_1",
                "Loc_2",
                "Loc_3",
                "Loc_4",
                "Loc_5",
                "Loc_6",
                "Loc_7",
                "Loc_8",
                "Loc_9",
                "Loc_10",
                "Loc_11",
                "Loc_12",
                "Loc_13",
                "Loc_14",
                "Loc_15",
                "Loc_16",
                "Loc_17",
                "Loc_18",
                "Loc_19",
                "Loc_20",
                "Loc_21",
                "Loc_22",
                "Loc_23",
                "Loc_24",
                "Loc_25",
                "Loc_26",
                "Loc_27",
                "Loc_28",
                "Loc_29",
                "Loc_30",
                "Loc_31",
                "Loc_32",
                "Loc_33",
                "Loc_34",
                "Loc_35",
                "Loc_36",
                "Loc_37",
                "Loc_38",
"Loc_39",
"Loc_40",
"Loc_41",
"Loc_42",
"Loc_43",
"Loc_44",
"Loc_45",
"Loc_46",
"Loc_47",
"Loc_48",
"Loc_49",
"Loc_50",
"Loc_51",
"Loc_52",
"Loc_53",
            "Loc_54",
            "Loc_55",
            "Loc_56",
            "Loc_57"]
    
    print("Number of names: ",len(names))
    print("Number of xs: ",len(xs))
    print("Number of ys: ",len(ys))
    return startyear, endyear, xs, ys, names


In [12]:


Kelvin = 273.15

#source file paths
dir_soiltemp_era5 = '/work/weather/era5monthly/soiltemp/'
dir_snow_cover_era5 = '/work/weather/era5monthly/snowdepth/'
dir_precip_era5 =  '/home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/precip/'
dir_temp_era5 = '/home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/temp'
dir_dewpointtemp_era5 = '/work/weather/era5monthly/dewpointtemp/'
dir_radiation_era5 = '/work/weather/era5monthly/radiation/'
dir_windu_era5 = '/home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/windu/'
dir_windv_era5 = '/home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/windv/'
#the following two are for Morten's bat project
dir_cloud_coverage_era5 = '/home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/cloud_coverage'
dir_cloudbaseheight_era5 = '/work/weather/era5monthly/cloudbaseheight/'
dir_atm_pressure_era5 = '/home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/atm_pressure/'

suffix_soiltemp_era5 = 'era5_soiltemp'
suffix_snowdepth_era5 = 'era5_snowdepth'
suffix_precip_era5 = 'era5_precip'
suffix_dewpointtemp_era5 = 'era5_dewpointtemp'
suffix_temp_era5 = 'era5_temp'
suffix_radiation_era5 = 'era5_radiation'
suffix_windu_era5 = 'era5_windu'
suffix_windv_era5 = 'era5_windv'
#the following two are for Morten's bat project
suffix_cloud_coverage_era5 ='era5_cloud_coverage'
suffix_cloudbaseheight_era5 ='era5_cloud_base_height'
suffix_atm_pressure_era5 ='era5_atm_pressure'

output_dir = '/home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/generated_data'

startyear, endyear, xs, ys, names = country_name_coordinate_year()

#
#change this to true if hourly data is needed
hourly_flag = True

##variable to store the loaded data
actual_df = []
#loading the data

#temperature, in K, minus Kelvin
temp = get_param_era5(dir_temp_era5, suffix_temp_era5, ys, xs, startyear, endyear, 'mean_temp', hourly_flag)
for i in range(len(xs)):
    temp[i]['mean_temp'] -= Kelvin
    temp[i]['mean_temp'] = temp[i]['mean_temp'].fillna(-99999)
    actual_df.append(temp[i])
    actual_df[i].insert(0, "year", actual_df[i].apply(lambda row: row['time'].year, axis=1), True)
    actual_df[i].insert(0, "month", actual_df[i].apply(lambda row: row['time'].month, axis=1), True)
    actual_df[i].insert(0, "day", actual_df[i].apply(lambda row: row['time'].day, axis=1), True)
    actual_df[i].insert(0, "hour", actual_df[i].apply(lambda row: row['time'].hour, axis=1), True)

#wind, u: direction of east; v: direction of north, based on these two, the wind is converted into wind speed and direction (from 0 to 2pi, staring from east and anticlockwise)
wind_u = get_param_era5(dir_windu_era5, suffix_windu_era5, ys, xs, startyear, endyear, 'wind_u', hourly_flag)
wind_v = get_param_era5(dir_windv_era5, suffix_windv_era5, ys, xs, startyear, endyear, 'wind_v', hourly_flag)
wind_speed = []
wind_direction = []
for i in range(len(xs)):
    wind_speed.append(wind_u[i].copy(deep=True))
    wind_speed[i].rename(columns={'wind_u':'wind_speed'}, inplace=True)
    wind_direction.append(wind_u[i].copy(deep=True))
    wind_direction[i].rename(columns={'wind_u':'wind_direction'}, inplace=True)
    wind_speed[i]['wind_speed'] = np.sqrt(np.power(wind_v[i]['wind_v'],2)+np.power(wind_u[i]['wind_u'],2))
    wind_direction[i]['wind_direction'] = np.arctan2(wind_v[i]['wind_v'], wind_u[i]['wind_u']) #east is the 0
    wind_direction[i].loc[wind_direction[i]['wind_direction']<0, 'wind_direction'] += 2*np.pi
    #wind_speed[i]['wind_speed'] = wind_speed[i]['wind_speed'].fillna(-99999)
    #wind_direction[i]['wind_direction'] = wind_direction[i]['wind_direction'].fillna(-99999)
    actual_df[i].insert(6, 'wind_speed', wind_speed[i]['wind_speed'], True)
    actual_df[i].insert(7, 'wind_direction', wind_direction[i]['wind_direction'], True)


#precipitation
precip = get_param_era5(dir_precip_era5, suffix_precip_era5, ys, xs, startyear, endyear, 'precip', hourly_flag)
for i in range(len(xs)):
    precip[i]['precip'] *= 1000 #change to mm
    precip[i].loc[precip[i]['precip']<0.0001, 'precip']=0.0
    #precip[i]['precip'] = precip[i]['precip'].fillna(-99999)
    actual_df[i].insert(8, 'precip', precip[i]['precip'], True)
    
# cloud_coverage (SQS)
cloud_coverage = get_param_era5(dir_cloud_coverage_era5, suffix_cloud_coverage_era5, ys, xs, startyear, endyear, 'cloud_coverage', hourly_flag)
for i in range(len(xs)):
    actual_df[i].insert(9, 'cloud_coverage', cloud_coverage[i]['cloud_coverage'], True)

# atm_pressure (SQS)
atm_pressure = get_param_era5(dir_atm_pressure_era5, suffix_atm_pressure_era5, ys, xs, startyear, endyear, 'atm_pressure', hourly_flag)
for i in range(len(xs)):
    actual_df[i].insert(10, 'atm_pressure', atm_pressure[i]['atm_pressure'], True)

# #soil Temperature
# soiltemp = get_param_era5(dir_soiltemp_era5, suffix_soiltemp_era5, ys, xs, startyear, endyear, 'soiltemp', hourly_flag)
# for i in range(len(xs)):
#     soiltemp[i]['soiltemp'] -= Kelvin
#     #soiltemp[i]['soiltemp'] = soiltemp[i]['soiltemp'].fillna(-99999)
#     actual_df[i].insert(9, 'soiltemp', soiltemp[i]['soiltemp'], True)

# #snow depth
# snowdepth = get_param_era5(dir_snow_cover_era5, suffix_snowdepth_era5, ys, xs, startyear, endyear, 'snowdepth', hourly_flag)
# for i in range(len(xs)):
#     snowdepth[i].loc[snowdepth[i]['snowdepth']<0, 'snowdepth'] = 0
#     snowdepth[i]['snowdepth'] *= 1000 #change to mm
#     #snowdepth[i]['snowdepth'] = snowdepth[i]['snowdepth'].fillna(-99999)
#     actual_df[i].insert(10, 'snowdepth', snowdepth[i]['snowdepth'], True)

# #humidity
# dewtemp = get_param_era5(dir_dewpointtemp_era5, suffix_dewpointtemp_era5, ys, xs, startyear, endyear, 'dewtemp', hourly_flag)
# humidity = []
# for i in range(len(xs)):
#     humidity.append(dewtemp[i].copy(deep=True))
#     humidity[i].rename(columns={'dewtemp':'humidity'}, inplace=True)
#     dewtemp[i]['dewtemp'] -= Kelvin
#     humidity[i]['humidity'] = 100 - 5*(temp[i]['mean_temp']-dewtemp[i]['dewtemp'])
#     #humidity[i]['humidity'] = humidity[i]['humidity'].fillna(-99999)
#     actual_df[i].insert(11, 'humidity', humidity[i]['humidity'], True)

# #radiation
# radiation = get_param_era5(dir_radiation_era5, suffix_radiation_era5, ys, xs, startyear, endyear, 'radiation', hourly_flag)
# for i in range(len(xs)):
#     radiation[i].loc[radiation[i]['radiation']<0.01, 'radiation']=0.0
#     #radiation[i]['radiation'] /= 3600.0
#     #radiation[i]['radiation'] = radiation[i]['radiation'].fillna(-99999)
#     actual_df[i].insert(12, 'radiation', radiation[i]['radiation'], True)


# #the following two are for Morten's bat project, please comment them out
# #cloudbaseheight
# """
# cloudbaseheight = get_param_era5(dir_cloudbaseheight_era5, suffix_cloudbaseheight_era5, ys, xs, startyear, endyear, 'cloudbaseheight', hourly_flag)
# for i in range(len(xs)):
#     #cloudbaseheight[i]['cloudbaseheight'] = cloudbaseheight[i]['cloudbaseheight'].fillna(-99999)    
#     actual_df[i].insert(9, 'cloudbaseheight', cloudbaseheight[i]['cloudbaseheight'], True)
# totalcloudcover = get_param_era5(dir_totalcloudcover_era5, suffix_totalcloudcover_era5, ys, xs, startyear, endyear, 'totalcloudcover', hourly_flag)
# for i in range(len(xs)):
#     #totalcloudcover[i]['totalcloudcover'] = totalcloudcover[i]['totalcloudcover'].fillna(-99999)
#     actual_df[i].insert(10, 'totalcloudcover', totalcloudcover[i]['totalcloudcover'], True)
# """
#processing the data.
for i in range(len(xs)):
    actual_df[i].drop('time', axis=1, inplace=True) # let us drop the time column before interpolating
    

    print("Looking for weather @ coordinates Lat: "+str(ys[i])+" Long: "+str(xs[i]))
    #actual_df[i]['SoilTemp']=actual_df[i]['SoilTemp']-Kelvin
    #actual_df[i]['SnowCover']=actual_df[i]['SnowCover']*100 #originally in m
    interpolated_data = interpolate_data(actual_df[i])
    write_weather(interpolated_data, os.path.join(output_dir, str(names[i])+"_era5.pre"))


Ignoring index file '/home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/temp/era5_temp_2023_01.grib.5b7b6.idx' older than GRIB file


Number of names:  57
Number of xs:  57
Number of ys:  57
reading era5_temp_2023_01.grib


/home/au472091/venv/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/tmp/ipykernel_1620840/3401802185.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  outdf[count]=pd.concat([outdf[count], out], axis=0, ignore_index=True)
/tmp/ipykernel_1620840/3401802185.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To

reading era5_temp_2023_02.grib


/home/au472091/venv/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
Ignoring index file '/home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/temp/era5_temp_2023_03.grib.5b7b6.idx' older than GRIB file


...

reading era5_atm_pressure_2025_03.grib


/home/au472091/venv/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


reading era5_atm_pressure_2025_04.grib


/home/au472091/venv/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


Looking for weather @ coordinates Lat: 55.6053 Long: 7.88513
Writing /home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/generated_data/Loc_1_era5.pre
Looking for weather @ coordinates Lat: 55.70201 Long: 8.05116
Writing /home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/generated_data/Loc_2_era5.pre
Looking for weather @ coordinates Lat: 55.79587 Long: 7.87969
Writing /home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/generated_data/Loc_3_era5.pre
Looking for weather @ coordinates Lat: 55.73477 Long: 7.79082
Writing /home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/generated_data/Loc_4_era5.pre
Looking for weather @ coordinates Lat: 55.88947 Long: 7.70791
Writing /home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/generated_data/Loc_5_era5.pre
Looking for weather @ coordinates Lat: 55.79229 Long: 7.5397
Writing /home/au472091/OneDrive/au/projects/pam_bats/analysis/data/weather/generated_data/Loc_6_era5.pre
